In [7]:
import numpy as np
import pandas as pd
import cufflinks
import json
import os
from copy import deepcopy
from datetime import datetime

import trading_gym
from trading_gym.ray.logger import calculate_tearsheet, CustomLogger, PPOTensorboard
from trading_gym.registry.gaia.v7.env import GAIAPredictorsContinuousV7
from trading_gym.ray.models import MultiLayersPerceptron

import ray
from ray import rllib
from ray import tune
from ray.rllib.models import ModelCatalog
from ray.rllib.models.model import Model
from ray.rllib.models.misc import normc_initializer, get_activation_fn
from ray import cloudpickle
from ray.utils import binary_to_hex, hex_to_binary

import tensorflow as tf
import tensorflow.contrib.slim as slim

ModelCatalog.register_custom_model(MultiLayersPerceptron.__name__, MultiLayersPerceptron)
cufflinks.go_offline()
ray.init(ignore_reinit_error=True)
trading_gym.__package__, trading_gym.__version__

2019-07-15 09:55:34,962	ERROR worker.py:1343 -- Calling ray.init() again after it has already been called.


('trading-gym', '0.7.6')

In [2]:
from ray.rllib.models import ModelCatalog
from ray.rllib.models.model import Model
from ray.rllib.models.misc import normc_initializer, get_activation_fn
import tensorflow as tf
import tensorflow.contrib.slim as slim


class MLP(Model):
    def _build_layers_v2(self, input_dict: dict, num_outputs: int, config: dict):
        import tensorflow.contrib.slim as slim

        with tf.name_scope("fc_net"):
            last_layer = input_dict['obs']
            activation = get_activation_fn(config.get("fcnet_activation"))
            for i, size in enumerate(config.get("fcnet_hiddens"), 1):
                last_layer = slim.fully_connected(
                    inputs=last_layer,
                    num_outputs=size,
                    weights_initializer=normc_initializer(1.0),
                    activation_fn=activation,
                    scope="fc{}".format(i),
                )
#                 We don't need any dropout at this stage
#                 last_layer = tf.layers.dropout(
#                     inputs=last_layer,
#                     rate=config['custom_options']["fcnet_dropout_rate"],
#                     training=input_dict['is_training'],
#                     name="dropout{}".format(i),
#                 )
            output = slim.fully_connected(
                inputs=last_layer,
                num_outputs=num_outputs,
                weights_initializer=normc_initializer(0.01),
                activation_fn=None,
                scope="fc_out",
            )
            return output, last_layer

ModelCatalog.register_custom_model(MLP.__name__, MLP)

In [3]:
def cloudpickleloads(obj):
    if isinstance(obj, dict):
        try:
            return cloudpickle.loads(hex_to_binary(obj["value"]))
        except:
            for key, value in obj.items():
                if isinstance(value, dict):
                    if sorted(value) == ['_type', 'value']:
                        obj[key] = cloudpickle.loads(hex_to_binary(value["value"]))
                    else:
                        obj[key] = cloudpickleloads(value)
                elif isinstance(value, list):
                    for i, item in enumerate(value):
                        obj[key][i] = cloudpickleloads(item)
    return obj

In [4]:
# Actually with transaction cost now (Default)
paths = {2007: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost/clip_0.8-tc-WalkForward-750k2007/experiment_state-2019-07-04_09-43-03.json',
        2008: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost/clip_0.8-tc-WalkForward-750k2008/experiment_state-2019-07-04_10-15-25.json',
        2009: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost/clip_0.8-tc-WalkForward-750k2009/experiment_state-2019-07-04_10-51-51.json',
        2010: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost/clip_0.8-tc-WalkForward-750k2010/experiment_state-2019-07-04_11-28-12.json',
        2011: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost/clip_0.8-tc-WalkForward-750k2011/experiment_state-2019-07-04_11-57-09.json',
        2012: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost/clip_0.8-tc-WalkForward-750k2012/experiment_state-2019-07-04_12-23-50.json',
        2013: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost/clip_0.8-tc-WalkForward-750k2013/experiment_state-2019-07-04_12-50-33.json',
        2014: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost/clip_0.8-tc-WalkForward-750k2014/experiment_state-2019-07-04_13-16-57.json',
        2015: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost/clip_0.8-tc-WalkForward-750k2015/experiment_state-2019-07-04_13-44-00.json',
        2016: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost/clip_0.8-tc-WalkForward-750k2016/experiment_state-2019-07-04_14-14-27.json',
        2017: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost/clip_0.8-tc-WalkForward-750k2017/experiment_state-2019-07-04_14-49-49.json'
        }

# With double trans costs
# paths = {2007: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_double/clip_0.8-tc-WalkForward-750k2007/experiment_state-2019-07-08_09-08-56.json',
#         2008: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_double/clip_0.8-tc-WalkForward-750k2008/experiment_state-2019-07-08_09-34-49.json',
#         2009: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_double/clip_0.8-tc-WalkForward-750k2009/experiment_state-2019-07-08_10-02-15.json',
#         2010: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_double/clip_0.8-tc-WalkForward-750k2010/experiment_state-2019-07-08_10-29-33.json',
#         2011: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_double/clip_0.8-tc-WalkForward-750k2011/experiment_state-2019-07-08_10-58-06.json',
#         2012: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_double/clip_0.8-tc-WalkForward-750k2012/experiment_state-2019-07-08_11-24-29.json',
#         2013: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_double/clip_0.8-tc-WalkForward-750k2013/experiment_state-2019-07-08_11-50-59.json',
#         2014: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_double/clip_0.8-tc-WalkForward-750k2014/experiment_state-2019-07-08_12-18-35.json',
#         2015: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_double/clip_0.8-tc-WalkForward-750k2015/experiment_state-2019-07-08_12-47-25.json',
#         2016: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_double/clip_0.8-tc-WalkForward-750k2016/experiment_state-2019-07-08_13-14-43.json',
#         2017: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_double/clip_0.8-tc-WalkForward-750k2017/experiment_state-2019-07-08_13-41-22.json'
#         } 

# Now triple trans costs
# paths = {2007: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2007/experiment_state-2019-07-08_14-26-15.json',
#         2008: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2008/experiment_state-2019-07-08_14-52-38.json',
#         2009: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2009/experiment_state-2019-07-08_15-20-13.json',
#         2010: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2010/experiment_state-2019-07-08_15-47-57.json',
#         2011: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2011/experiment_state-2019-07-08_16-15-37.json',
#         2012: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2012/experiment_state-2019-07-08_16-44-41.json',
#         2013: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2013/experiment_state-2019-07-08_17-13-04.json',
#         2014: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_triple/clip_0.8-tc-WalkForward-750k2014/experiment_state-2019-07-08_17-39-13.json',
#         2015: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_double/clip_0.8-tc-WalkForward-750k2015/experiment_state-2019-07-08_12-47-25.json',
#         2016: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_double/clip_0.8-tc-WalkForward-750k2016/experiment_state-2019-07-08_13-14-43.json',
#         2017: '/home/Nicholas/trading-gym/notebooks/registry/gaia/v8/logs/tran_cost_double/clip_0.8-tc-WalkForward-750k2017/'
#         } 

In [5]:
policies = dict()
env_configs = dict()
for year, path in paths.items():
    print(year)
    with open(path) as f:
        metadata = json.load(f)
        checkpoints = metadata['checkpoints']
        runner_data = metadata['runner_data']
        stats = metadata['stats']

    for checkpoint in checkpoints:
        checkpoint = cloudpickleloads(checkpoint)
        cp = cloudpickle.loads(hex_to_binary(checkpoint['_checkpoint']))
        config = checkpoint['config']
        env_cls = config['env']
        env_config = config['env_config']
        path_restore = os.path.join(checkpoint['logdir'], cp.value)
        

    agent = rllib.agents.ppo.PPOTrainer(config, env_cls)
    agent.restore(path_restore)
    
    policies[year] = agent.get_policy()
    env_configs[year] = env_config

2007


Exception: Unknown config parameter `local_evaluator_tf_session_args` 

In [ ]:
levels = list()
mappings = pd.DataFrame()
mapping_functions = dict()
for year, policy in policies.items():
    print('_______________________________________{}____________________________________________'.format(year))
    
    env_config = env_configs[year]
    env = env_cls(env_config)
    episode = env.sample_episode(
        fold='test-set',
        policy=policy,
    )
    renderer = env.render(
        benchmark=env._load_benchmark().squeeze().loc[str(year)],
        risk_free=env._load_risk_free().squeeze().loc[str(year)],
    )
    renderer.plotly_report()
    renderer.tearsheet()
    
    levels.append(renderer.level.to_frame().pct_change())
    
    
    
    actions = pd.DataFrame(episode.actions, index=episode.time, columns=env.action_space.contracts[1:])
    states = pd.DataFrame(episode.states, index=episode.time)
    target_russell = actions[actions.columns[0]]
    mapping = states[0].to_frame('GAIA Predictor').join(target_russell)
    mappings = mappings.append(mapping)
    
    
    
    this_mapping = dict()
    env.reset()
    for predictor in np.linspace(-0.01, 0.01, 500):
        from trading_gym.registry.gaia.v7.env import EventGAIAPredictor
        from scipy.special import softmax
        event = EventGAIAPredictor(time=datetime.now(), equities=predictor)
        env.add_event(event)
        state = env.state.get_values()

        _, _, info = policy.compute_single_action(state, [])
        logits = info['behaviour_logits']
        action = softmax(logits)
        target_weight_equity = action[0]
        this_mapping[predictor] = target_weight_equity
    mapping_functions[year] = this_mapping

In [ ]:

mappings.sort_index().iplot(
#     secondary_y='Russell 1000'  ---    This was here before
    secondary_y=str(mappings.columns[-1]),
    vline=[str(year) for year in range(2008, 2019)]
)

In [ ]:
pd.DataFrame(mapping_functions).iplot(title='Mapping functions trained with deep reinforcement learning', yTitle='Target weight for Russell 1000', xTitle='Standardized GAIA predictor')

In [ ]:
mappings.set_index('GAIA Predictor').iplot(kind='scatter', mode='markers', size=3, yTitle='Target weight for Russell 1000', xTitle='Standardized GAIA predictor')